In [76]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import cloudscraper
import warnings
warnings.filterwarnings("ignore")

In [77]:
stories_urls_df = pd.read_csv("C:\\Users\\annieboadu\\OneDrive - Affinity Africa\\Documents\\ghana-web-data-main\\CSV files\\Selected_Sections_data.csv")

In [78]:
abroad_gh = stories_urls_df[stories_urls_df['Section'] == "Abroad, Ghanaians"]

## Getting story URLS

In [79]:
df_columns = ['Section Title','Url','Url Extracted']

In [80]:
def get_story_urls(row):
    # try:
    df = pd.DataFrame()
    scraper = cloudscraper.create_scraper()
    page = scraper.get(row["url"])
    soup = BeautifulSoup(page.content, 'html.parser')
    page_stories = soup.find_all("div", class_ = "afcon-news list")
    for ul in page_stories:
        for li in ul.findAll('li'):
            data = li.find("a")
            url = row['url']+data.get('href')
            url_extracted = 0
            temp_row = pd.DataFrame(data = [[row["Section"], url,url_extracted]], columns = df_columns)
            df = pd.concat([df,temp_row], ignore_index=True)  
        time.sleep(20)
    # except:
    #     pass

    return df

In [81]:
url_df  = abroad_gh.apply(lambda row : get_story_urls(row),axis = 1)

In [82]:
article_urls_df = pd.DataFrame(columns = df_columns)

In [83]:
for row in url_df:
    article_urls_df = article_urls_df.append(row)

In [84]:
article_urls_df.to_csv("C:\\Users\\annieboadu\\OneDrive - Affinity Africa\\Documents\\ghana-web-data-main\\CSV files\\Abroad Ghanaians Urls.csv")

In [85]:
##push to db

## Extracting Stories from urls

In [86]:
story_df_columns = ['Section','Header','Date','Title','Information']
stories_data = pd.DataFrame(columns = story_df_columns)

In [87]:
def scrape_song_lyrics(url):
    page = requests.get(url)
    html = BeautifulSoup(page.text, 'html.parser')
    
    try:
        lyrics = html.find('div', class_='lyrics').get_text()
    except AttributeError:
        print('cannot find the target div: lyrics')
        return None
    
    else:
        #remove identifiers like chorus, verse, etc
        lyrics = re.sub(r'[\(\[].*?[\)\]]', '', lyrics)
        #remove empty lines
        lyrics = os.linesep.join([s for s in lyrics.splitlines() if s]) 
        return lyrics

In [88]:
def get_abroad_gh_articles(section_title,url):
    df = pd.DataFrame()
    scraper = cloudscraper.create_scraper()
    page = scraper.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    page_stories = soup.find(id = "medsection1")
    try:
        div_class = page_stories.find("div", class_ = "article-left-col")       
    except AttributeError:
        print('cannot find the target div: article-left-col')
        file_object = open("C:\\Users\\annieboadu\\OneDrive - Affinity Africa\\Documents\\ghana-web-data-main\\Error Urls\\article urls.txt", 'a')
        file_object.write(f'\n\n{section_title},{url}')
        file_object.close()
        return None
    else:
        header = div_class.find('p', class_ = "floatLeft").text
        date = div_class.find('a', id = "date").text
        title = div_class.find('h1').text
        info = div_class.find('p',id = "article-123").text
        temp_row = pd.DataFrame(data = [[section_title, header, date, title, info]], columns = story_df_columns)
        df = pd.concat([df,temp_row],ignore_index =True)
    return df

In [89]:
abroad_gh_final_df = article_urls_df.apply(lambda row:get_abroad_gh_articles(row['Section Title'],row['Url']), axis = 1)

cannot find the target div: article-left-col
cannot find the target div: article-left-col
cannot find the target div: article-left-col
cannot find the target div: article-left-col
cannot find the target div: article-left-col
cannot find the target div: article-left-col


In [90]:
abroad_stories_urls_df = pd.DataFrame(columns = story_df_columns)

In [91]:
for row in abroad_gh_final_df:
    abroad_stories_urls_df = abroad_stories_urls_df.append(row)

In [92]:
abroad_stories_urls_df.reset_index(inplace=True)

In [93]:
del abroad_stories_urls_df["index"]

In [94]:
abroad_stories_urls_df.head()

,Section,Header,Date,Title,Information
0,"Abroad, Ghanaians","Diasporia News of Sunday, 3 July 2022",2022-07-03,Hands Across Africa Team calls on Diaspora Aff...,The Hands Across Africa team from the Caribbea...
1,"Abroad, Ghanaians","Diasporia News of Friday, 1 July 2022",2022-07-01,NPP UK appoints executives for the next 4 years,"On Sunday 26th June 2022, the Chairman of NPP ..."
2,"Abroad, Ghanaians","Diasporia News of Friday, 1 July 2022",2022-07-01,"Foreign investors are ‘using’ Ghana, but citiz...",Norwegian and University of Ghana Alumni Birte...
3,"Abroad, Ghanaians","Diasporia News of Friday, 1 July 2022",2022-07-01,My ex-wife’s attitude changed after I brought ...,Denmark-based Ghanaian Issac Baidoo has reveal...
4,"Abroad, Ghanaians","Diasporia News of Thursday, 30 June 2022",2022-06-30,A patient head-butted me and caused a brain in...,US-based Ghanaian Mr Phillip Ofori Yentumi has...
